In [15]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import numpy as np

In [2]:
data = pd.read_csv('clean_data.csv')
data_normalized = pd.read_csv('normalized_data.csv')

data = data.set_index('encounter_id')
data_normalized = data_normalized.set_index('encounter_id')

In [3]:
train, test = train_test_split(data_normalized, test_size=0.2)

In [4]:
#osetrenie future leak-u

train_target = train['readmitted']
test_target = test['readmitted']

train = train.drop(columns=['readmitted'])
test = test.drop(columns=['readmitted'])

In [5]:
gnb = GaussianNB()
y_pred = gnb.fit(train, train_target).predict(test)


In [6]:
print(metrics.classification_report(test_target, y_pred))

             precision    recall  f1-score   support

        0.0       0.74      0.02      0.03     10974
        1.0       0.46      0.99      0.63      9379

avg / total       0.61      0.47      0.31     20353



In [7]:
from sklearn.tree import DecisionTreeClassifier

In [8]:
clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=0)

dtc_pred = clf.fit(train, train_target).predict(test)

In [9]:
print(metrics.classification_report(test_target, dtc_pred))

             precision    recall  f1-score   support

        0.0       0.59      0.58      0.58     10974
        1.0       0.51      0.52      0.52      9379

avg / total       0.55      0.55      0.55     20353



In [13]:
clf.tree_.max_depth

51

In [25]:
%%time

parameters = {'max_depth' : np.arange(3, 52), 'min_samples_leaf' : np.arange(1, 11)}
tree = GridSearchCV(DecisionTreeClassifier(), parameters)

tree.fit(train, train_target)

Wall time: 49min 33s


In [26]:
tree_pred = tree.predict(test)

print(metrics.classification_report(test_target, tree_pred))

             precision    recall  f1-score   support

        0.0       0.62      0.75      0.68     10974
        1.0       0.62      0.46      0.53      9379

avg / total       0.62      0.62      0.61     20353



In [29]:
tree.best_params_

{'max_depth': 6, 'min_samples_leaf': 10}

In [31]:
dtc2 = DecisionTreeClassifier(max_depth=6, min_samples_leaf=10)

dtc2.fit(train, train_target)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [32]:
dtc2.feature_importances_


array([0.00000000e+00, 0.00000000e+00, 4.54899048e-02, 4.96327754e-03,
       1.21072031e-02, 6.65901476e-03, 1.29665980e-02, 2.99536630e-02,
       5.89123768e-02, 6.80777993e-01, 9.24790835e-02, 7.76537024e-04,
       1.24380981e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.09787360e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.99470146e-02,
       0.00000000e+00, 7.21923665e-03, 0.00000000e+00, 1.36736738e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.02450940e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.42459749e-04, 2.69121479e-03])

In [60]:
dtc_importances = pd.DataFrame()

dtc_importances = dtc_importances.assign(features = pd.Series(train.columns))
dtc_importances = dtc_importances.assign(importances = pd.Series(dtc2.feature_importances_))


In [61]:
dtc_importances

,features,importances
0,race,0.000000
1,gender,0.000000
2,age,0.045490
3,time_in_hospital,0.004963
4,num_lab_procedures,0.012107
5,num_procedures,0.006659
6,num_medications,0.012967
7,number_outpatient,0.029954
8,number_emergency,0.058912
9,number_inpatient,0.680778
